In [1]:
""" DataSet Description """

' DataSet Description '

In [47]:
""" Importing the libraries """

import numpy as np
import math
import re
import time
from google.colab import drive

try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [48]:
""" Data Pre-Processing :

1. Loading the files

"""

with open("/content/drive/My Drive/SelfPractise/DeepNLP/Transformer/europarl-v7.fr-en.en",
          mode='r',
          encoding='utf-8') as f:
    europarl_en = f.read()
with open("/content/drive/My Drive/SelfPractise/DeepNLP/Transformer/europarl-v7.fr-en.fr",
          mode='r',
          encoding='utf-8') as f:
    europarl_fr = f.read()
with open("/content/drive/My Drive/SelfPractise/DeepNLP/Transformer/P85-Non-Breaking-Prefix.en",
          mode='r',
          encoding='utf-8') as f:
    non_breaking_prefix_en = f.read()
with open("/content/drive/My Drive/SelfPractise/DeepNLP/Transformer/P85-Non-Breaking-Prefix.fr",
          mode='r',
          encoding='utf-8') as f:
    non_breaking_prefix_fr = f.read()

In [49]:
"""" 2. Cleaning the data 

I will split them in order to have the clean list of words and add space before each of them

and dot after each of them because It will helps us to clean the data properly """


non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]
non_breaking_prefix_fr = non_breaking_prefix_fr.split("\n")
non_breaking_prefix_fr = [' ' + pref + '.' for pref in non_breaking_prefix_fr]

In [51]:
""" 3. I will remove the points which are not full stop, I am removing unwnated points or dots to make it
easier for our algorithm to understand what really is it, Let's say we got some sentence where we have class starts

at 2 A.M in such cases, I will remove it and I will store it into a new corpus,

I will write the function to find the dots or points which are not at the end of the sentence

"""


corpus_en = europarl_en
# Add $$$ after non ending sentence points
for prefix in non_breaking_prefix_en:
    corpus_en = corpus_en.replace(prefix, prefix + '$$$')

    """ 
    Now I will check for the dots which are not followed by the space
        If the full stop is followed by any number of letters then it is not considered Full stop 
        Here I will start with re.sub then type of string, We are looking for,
        1. We will add points but will add backslash because point has a specific meaning in REGEX
        2. Then Any sets of letters or number which is right after it and This one begins with ? = which means
        it has to follow the points , Here = Means look for the letters or numbers but do not replace it
        3. As I am looking for any number hence 0-9 
        4. or , Please note , In REGEX , or is represented by | (vertical bar)
        5. any letters small or capital hence written a-z | A-Z 
        6. But our aim is to replace the points only hence will mention here only consider the points for the
           replacement part
        7. Then at the end , I will specify the string, I would like to apply to, In my case here it is 
            corpus eng hence put that here 
            
         NOw here we are I have selected all the points which i do not want to be ending points
         Let's remove those points
    """




corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en)

""" Second regex is for removing these points or get replaced with nothing because we want to remove them

"""


corpus_en = re.sub(r".\$\$\$", '', corpus_en)

""" Now There is posibility that There could be many white spaces 
1. I am saying to look for white space followed by white space + (LOok closely I have given two spaces) 
2. I am writing to say that I want to replace all those whites spaces with a single white space(Look I have given
    one single white spaces and at the end our corpus which is corpus_En

"""

corpus_en = re.sub(r"  +", " ", corpus_en)

""" NOw Let's split the corpus english with respect blacklash and n

"""

corpus_en = corpus_en.split('\n')

""" Now We will do the same with french corpus too

"""

corpus_fr = europarl_fr
for prefix in non_breaking_prefix_fr:
    corpus_fr = corpus_fr.replace(prefix, prefix + '$$$')
corpus_fr = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_fr)
corpus_fr = re.sub(r".\$\$\$", '', corpus_fr)
corpus_fr = re.sub(r"  +", " ", corpus_fr)
corpus_fr = corpus_fr.split('\n')

In [52]:
""" 4. Tokenizing the text, Tokenizing is the process of transforming a set of chracters into one corresponding

number or token

Here i will tfds library to tokenize the word 

Note : It is always advisible to have lower number of target_vocab_size, It helps in improving the performance

of transformer

"""


tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_en, target_vocab_size=2**13)


tokenizer_fr = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_fr, target_vocab_size=2**13)



In [54]:
""" 
I will create two GLobal variable which I will use later
1. Vocab size for English Language, 

I am giving value 2 here with plus sign because these are the words which will be added which means 2 words

will be added but they are really words, They are just Homemade, This will be token which will be added at the

begining of the sentence and other one will be added at the end of the sentence, As I said before, They are not

really words but will be considered for the better of our model and Will do the same thing for french language

"""


VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2 # = 8190
VOCAB_SIZE_FR = tokenizer_fr.vocab_size + 2 # = 8171


""" 
Here I will create the real inputs and outputs to our model, Input will be a list of sentence, Each 

sentence being a sequence of numbers corressponding to the encoding of words and I will use starting and ending tokens
so each sentence will start with the starting token that will be vocab size -2  which is exactly the vocab size of 

original tokenizer,

Tokenizer_en_encode  is the string sentence into a list of numbers according to the tokenizing creation that

I have already found. So I encode a sentence and add the end of our homemade ending token vocab size english-1

and This is for every sentence in our original corpus,

Now input is just a list of encoded sentences and will do the same with other language,

Our second langauge is the OUTPUT Because We are building here a translator

"""


inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in corpus_en]
outputs = [[VOCAB_SIZE_FR-2] + tokenizer_fr.encode(sentence) + [VOCAB_SIZE_FR-1]
           for sentence in corpus_fr]

In [55]:
""" 

5. IF you will look at our dataset, It has some really long sentence and this is the issue here is why

1. First one is when we will pads all sentences, So want to make all sentences the same lines for the batch to work

2. It will take long time to train if we keep the really sentences and We reallu don't need to have those long

sentences But IN case we are building a translator and have high power Machine, We can set the long sentences


Here I am giving maximum length to the sentence as 20,

Well! That seems quite low dataset


"""



MAX_LENGTH = 20

""" 

Creating a variable which will contain all the indices we want to get rid of 

Enumerate function is same as going through all the inputs but instead of just giving the sentences once at a time,

it gives the sentences and counts so first element will zero and the sentence second sentences will give one and so on

"""
idx_to_remove = [count for count, sent in enumerate(inputs)
                 if len(sent) > MAX_LENGTH]

""" 

Here I am doing reverse element, Now let's undertand why am i doing reverse element, let's say we want to remove
first and second element, Once we have removed second elements all the indices of the element after those one would be shifted
by one because these one would be  will have disappeared hence we need to process in the reverse order so that we are removing right
elements

And there I have used dell function which allows us to remove elements according to their indices right here
and I will do it for input and output because I want to match every elements in input with output

OKay So now I Have removed all the sentences from english that were longer than 20 words
and Will do the same for french too


"""

for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]
idx_to_remove = [count for count, sent in enumerate(outputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]



In [56]:
""" 

Input/ Output creation :


1. We will pad all our input and output which means we want to make each sentence of same length 

2. Want to create dataset to shuffle back and do things which are neccessory

I am padding POST which means I am padding zero at the end of each sentence  and Maxlength will tell How long

our sentences will be 

Will do the same thing with output 

"""


inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=MAX_LENGTH)


""" 

Now I will create the Dataset

Will decide the Batch_size, We want to use- I will take here 64 as our batch size, But later we can change it

Shuffle siz = 20K

"""

BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

""" 

NOw WE have dataset but in order to make it fully usefull or optimize it  by our training phase, WIll use 

first dataset cache- This helps in improving the way data is stored, They way we can have access to the data

during the training, It increases the speed of the training but has nothing to do with performance of the model

NOte : It is not mandatory to use hence you can ignore it, 

Dataset.prefetch is also used to get the training faster

"""


dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# Positional Encoding

In [57]:
""" 

This is very important part of Transformer , Now Will start building the transformer model

I will not write here my own embedding because it is already available here But will create a customer layer

in order to process this positional encoding that they explained in the Official paper , 

How positional encoding works , As I will be creating a custom layer hence it is going to be a class


Here I am creating a class positionalencoding which will be inherit from layers.layer class

"""


class PositionalEncoding(layers.Layer):


    """ 
    Whenever we are creating a custom layer or class we have to define a init method which is, each time you
    create an object from this class, It will take it as an agurments "Self" just like any other method, 
    This is the layer which will make only mathematical computation hence won't need any more parameters 
    because there will no trainable weights, No sub layers or anything this kind at this position encoding phase
    SO This init method will only call super positional encoding

    """

    def __init__(self):
        """ 
        
        This line will call the init function of the class layer which has been well called the right there
        so that the object can be built correctly with all the features that any layer can have

        """
        super(PositionalEncoding, self).__init__()
        """
        
        Now I will create another method that will perform the operation which is inside the sin and cosine function
        to understand that You can visit the official paper of Transformer 
        SO  here defining this method get_angel which accepts the parameters like
        1. self
        2. pos
        3. i
        4. d_model 
           Here I am just writing the formula which is used in transformer for positional encoding and will add it
           to our input 
        """
    def get_angles(self, pos, i, d_model):
        angles = 1 / np.power(10000., (2*(i//2)) / np.float32(d_model))
        return pos * angles
    def call(self, inputs):
        seq_length = inputs.shape.as_list()[-2]
        d_model = inputs.shape.as_list()[-1]
        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],
                                 np.arange(d_model)[np.newaxis, :],
                                 d_model)
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        pos_encoding = angles[np.newaxis, ...]
        return inputs + tf.cast(pos_encoding, tf.float32)

# Attention Machanism 

In [58]:
""" 

1. Scaled Dot product Attention  

NOte : If you want to understand how am i writing this, GO through the Transformer Research Paper

Here Mask's work is that it does not allow decoder to see the next word

"""


def scaled_dot_product_attention(queries, keys, values, mask):
    product = tf.matmul(queries, keys, transpose_b=True)
    
    keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32)
    scaled_product = product / tf.math.sqrt(keys_dim)
    
    if mask is not None:
        scaled_product += (mask * -1e9)
    
    attention = tf.matmul(tf.nn.softmax(scaled_product, axis=-1), values)
    
    return attention

In [59]:
"""

Multi Head Attention Machanism

Look at the architecture in the Research paper 

Here I will apply a linear function which is exactly a dense layer, Then SPlit the results into sub spaces

For Each of those sub spaces, I Will apply the scale dot product attention And then I Will concatinate it 

so that we can get back to the words that have the right dimension which is the model

And then Finally I will apply this last linear function to the whole computation

Here I am writing a class which will be inherits from layers.layers And It will take init python function which will accept self,
And Number of Projection we want


"""



class MultiHeadAttention(layers.Layer):
    
    def __init__(self, nb_proj):
        super(MultiHeadAttention, self).__init__()
        self.nb_proj = nb_proj
        
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.nb_proj == 0
        
        self.d_proj = self.d_model // self.nb_proj
        
        self.query_lin = layers.Dense(units=self.d_model)
        self.key_lin = layers.Dense(units=self.d_model)
        self.value_lin = layers.Dense(units=self.d_model)
        
        self.final_lin = layers.Dense(units=self.d_model)
        
    def split_proj(self, inputs, batch_size): # inputs: (batch_size, seq_length, d_model)
        shape = (batch_size,
                 -1,
                 self.nb_proj,
                 self.d_proj)
        splited_inputs = tf.reshape(inputs, shape=shape) # (batch_size, seq_length, nb_proj, d_proj)
        return tf.transpose(splited_inputs, perm=[0, 2, 1, 3]) # (batch_size, nb_proj, seq_length, d_proj)
    
    def call(self, queries, keys, values, mask):
        batch_size = tf.shape(queries)[0]
        
        queries = self.query_lin(queries)
        keys = self.key_lin(keys)
        values = self.value_lin(values)
        
        queries = self.split_proj(queries, batch_size)
        keys = self.split_proj(keys, batch_size)
        values = self.split_proj(values, batch_size)
        
        attention = scaled_dot_product_attention(queries, keys, values, mask)
        
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        
        concat_attention = tf.reshape(attention,
                                      shape=(batch_size, -1, self.d_model))
        
        outputs = self.final_lin(concat_attention)
        
        return outputs

# Encoder 

In [60]:
""" 

Let's look at the plan or steps I Will be performing to build our first encoder

1. Apply multi head attention

2. And then Add the result to the input and

3. Apply some regular linear function USING THE dense layer and again use these small add and known sequence at the end

"""


""" 


Now We will start by class EncoderLayer

Then Define the __init__ function, self as usual 

And As input We will have is Feed Forward units, This Feed forward units are number of units which I will use in neural network

Number of Projections : It is for the Attention machanism
And then Dropout Rate that I will to control the overfitting 

then we call to Super which will be applied to encoder layer and self and call the init Method from it


"""



class EncoderLayer(layers.Layer):
    
    def __init__(self, FFN_units, nb_proj, dropout_rate):
        super(EncoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout_rate = dropout_rate



        """ 
       
            I will write the build method for dimension of the models and things when we create the object at encoder layer


        """

    def build(self, input_shape):
        self.d_model = input_shape[-1]

        """ 
    
        Here I am aplying multi head attention to the input with number of projection
        Will add some Drop out and normalization layer with few Dense layer, 
        It is good to go through the Model's architecture while writing code for this
        """




        self.multi_head_attention = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)
        
        self.dense_1 = layers.Dense(units=self.FFN_units, activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_2 = layers.Dropout(rate=self.dropout_rate)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, mask, training):
        attention = self.multi_head_attention(inputs,
                                              inputs,
                                              inputs,
                                              mask)
        attention = self.dropout_1(attention, training=training)
        attention = self.norm_1(attention + inputs)
        
        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_2(outputs, training=training)
        outputs = self.norm_2(outputs + attention)
        
        return outputs

In [61]:
class Encoder(layers.Layer):
    
    def __init__(self,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 vocab_size,
                 d_model,
                 name="encoder"):
        super(Encoder, self).__init__(name=name)
        self.nb_layers = nb_layers
        self.d_model = d_model
        
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.enc_layers = [EncoderLayer(FFN_units,
                                        nb_proj,
                                        dropout_rate) 
                           for _ in range(nb_layers)]
    
    def call(self, inputs, mask, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)
        
        for i in range(self.nb_layers):
            outputs = self.enc_layers[i](outputs, mask, training)

        return outputs

# Decoder

In [62]:
"""
 
In decoder part,  In decoder, Everything is same but only difference is that in Decoder, We use the Attention machanism twice

Will start by creating the Sub layer which would be named Decoder sub layer which will have argument layers.layer


and then use init function after a class as usual

self,

Feedforward units

number of projections

Dropout rates

It is almost same as encoder not much difference 

"""



class DecoderLayer(layers.Layer):
    
    def __init__(self, FFN_units, nb_proj, dropout_rate):
        super(DecoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout_rate = dropout_rate
    
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        
        # Self multi head attention
        self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)
        
        # Multi head attention combined with encoder output
        self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
        self.dropout_2 = layers.Dropout(rate=self.dropout_rate)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)
        
        # Feed foward
        self.dense_1 = layers.Dense(units=self.FFN_units,
                                    activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_3 = layers.Dropout(rate=self.dropout_rate)
        self.norm_3 = layers.LayerNormalization(epsilon=1e-6)
        
    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        attention = self.multi_head_attention_1(inputs,
                                                inputs,
                                                inputs,
                                                mask_1)
        attention = self.dropout_1(attention, training)
        attention = self.norm_1(attention + inputs)
        
        attention_2 = self.multi_head_attention_2(attention,
                                                  enc_outputs,
                                                  enc_outputs,
                                                  mask_2)
        attention_2 = self.dropout_2(attention_2, training)
        attention_2 = self.norm_2(attention_2 + attention)
        
        outputs = self.dense_1(attention_2)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_3(outputs, training)
        outputs = self.norm_3(outputs + attention_2)
        
        return outputs



In [63]:
class Decoder(layers.Layer):
    
    def __init__(self,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 vocab_size,
                 d_model,
                 name="decoder"):
        super(Decoder, self).__init__(name=name)
        self.d_model = d_model
        self.nb_layers = nb_layers
        
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout_rate)
        
        self.dec_layers = [DecoderLayer(FFN_units,
                                        nb_proj,
                                        dropout_rate) 
                           for i in range(nb_layers)]
    
    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)
        
        for i in range(self.nb_layers):
            outputs = self.dec_layers[i](outputs,
                                         enc_outputs,
                                         mask_1,
                                         mask_2,
                                         training)

        return outputs

# Transformer

In [64]:
"""

I have already made the parts of transformer

Encoder and Decoder but few things were left there like Mask function within encoder 

and few things in decoder, Which I will be doing here


"""


class Transformer(tf.keras.Model):
    """

    Vocab size is here because we need embeddings and this would be for encoder and other vocab size would be for decoder
    d_model = which is dimension of the model which we will have at the end of the embedding
    nb_layers = Number of layers, which means number of times we will apply the encoder and decoder layers, In the paper they said Six but I will try with other numbers too

    FFN_Units = number of units inside Feed forward network

    nb_proj = number of projections inside multihead attention layer
    dropout_rate = to control overfitting
    name = will give name to the model

    """
    def __init__(self,
                 vocab_size_enc,
                 vocab_size_dec,
                 d_model,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 name="transformer"):
        super(Transformer, self).__init__(name=name)
        
        self.encoder = Encoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout_rate,
                               vocab_size_enc,
                               d_model)
        self.decoder = Decoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout_rate,
                               vocab_size_dec,
                               d_model)
        self.last_linear = layers.Dense(units=vocab_size_dec, name="lin_ouput")


        """ 

        
          Now,This part is all about dealing with keys, values or queries in the matrix. We can have a look at
          the research paper of the transformer . I Will create the masking function which will accept the sequence of input (dataset), 
          here tf.newaxis is just changing to the new Dimension, And Now Why did I do that, Look back at previous code while

          creating the multihead attention machanism, I Had split the dimensions of the embedding which means that we have

          we have an additional dimension, Look at the number of projections of the attention machanism, Now creating this

          New dimension so that it can be applied to each of the projections 

        """
    
    def create_padding_mask(self, seq):
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
        return mask[:, tf.newaxis, tf.newaxis, :]

        """

             NOW, I will be creating not just a vector but a complete full matrix, and The Goal of this matrix is to 
             prevent it seeing the future words and best way to do is to use Triangular matrix is use this method from 
             tensorflow which is linalg.band_part. This Tringular matrix is only half top right part of the matrix, It is
             in a tringular shape and does not have zeros
    
             This function will have self and seq as parameters then shape of the sequence as statement And As I have already
            mentioned above, Will be creating a tringular matrix using this linal.band_part tensorflow method so what I am 
             doing here is adding the zero at the top right parts of this matrix. This function linal.band_part add condition
             on the indices of all matrix and put zeros if those conditions are completed
    
             So Basically the first part says if the I in row and J in columns in all matrix, This add the condition about
             how I supposed to  be higher than J
    
             Let me explain this I and J here. SO word I will not have access to word J, I want to mask the word that will be
    
             in the future and that is the reason I am 1- the whole matrix(means masking the future words here)

        """

    def create_look_ahead_mask(self, seq):
        seq_len = tf.shape(seq)[1]
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return look_ahead_mask

        """

            Here I will combine the encoder and decoder in the Call method , We get input for the encoder and input for
            the decoder and information about whether we are training it or not
    
            And Will create the mask in the encoder, This will be a simple padding mask because Here transformer really dont
            need look ahead mask
        """
    
    def call(self, enc_inputs, dec_inputs, training):
        enc_mask = self.create_padding_mask(enc_inputs)
        dec_mask_1 = tf.maximum(
            self.create_padding_mask(dec_inputs),
            self.create_look_ahead_mask(dec_inputs)
        )
        dec_mask_2 = self.create_padding_mask(enc_inputs)
        
        enc_outputs = self.encoder(enc_inputs, enc_mask, training)
        dec_outputs = self.decoder(dec_inputs,
                                   enc_outputs,
                                   dec_mask_1,
                                   dec_mask_2,
                                   training)
        
        outputs = self.last_linear(dec_outputs)
        
        return outputs


# Training


In [65]:
"""

Will creat the model, and defining the hyper parameters

I will have explained almost every parameters before hence won't be writing again


NB_Layers should be as lower as possible because it makes your translator perfect


Number_Projection : It is number of head

"""

tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 128 # 512
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8 # 8
DROPOUT_RATE = 0.1 # 0.1

"""

Let's build the transformer
Will put the hyper parameters there

"""
transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_FR,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout_rate=DROPOUT_RATE)

In [66]:
"""

At this step, Will define the loss function 

"""

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")

def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss_ = loss_object(target, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

In [67]:
""" 
Designing the Optimizer part
Designing my own learning rate
"""


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

leaning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(leaning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)
        

In [68]:
""" 

This check point is important in starting the model at every epochs or whatever number we can give to start the model from

Will save it in my same google drive which I mounted

WIll create a checkpoint manager , Work of Checkpoint manager is that

If we have 5 check points, Can manage the check points, TO say if we have given maximum 5 check points. it can start

new checkpoints after 5 

"""


checkpoint_path = "./content/drive/My Drive/SelfPractise/DeepNLP/Transformer/ckpt/"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [69]:

EPOCHS = 10
for epoch in range(EPOCHS):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for (batch, (enc_inputs, targets)) in enumerate(dataset):
        dec_inputs = targets[:, :-1]
        dec_outputs_real = targets[:, 1:]
        with tf.GradientTape() as tape:
            predictions = transformer(enc_inputs, dec_inputs, True)
            loss = loss_function(dec_outputs_real, predictions)
        
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
        
        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)
        
        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))
            
    ckpt_save_path = ckpt_manager.save()
    print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                        ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

Start of epoch 1
Epoch 1 Batch 0 Loss 6.3560 Accuracy 0.0000
Epoch 1 Batch 50 Loss 6.2348 Accuracy 0.0051
Epoch 1 Batch 100 Loss 6.2013 Accuracy 0.0269
Epoch 1 Batch 150 Loss 6.1185 Accuracy 0.0338
Epoch 1 Batch 200 Loss 6.0337 Accuracy 0.0367
Epoch 1 Batch 250 Loss 5.9359 Accuracy 0.0434
Epoch 1 Batch 300 Loss 5.8091 Accuracy 0.0511
Epoch 1 Batch 350 Loss 5.6797 Accuracy 0.0571
Epoch 1 Batch 400 Loss 5.5571 Accuracy 0.0616
Epoch 1 Batch 450 Loss 5.4353 Accuracy 0.0654
Epoch 1 Batch 500 Loss 5.3237 Accuracy 0.0703
Epoch 1 Batch 550 Loss 5.2190 Accuracy 0.0755
Epoch 1 Batch 600 Loss 5.1232 Accuracy 0.0808
Epoch 1 Batch 650 Loss 5.0313 Accuracy 0.0858
Epoch 1 Batch 700 Loss 4.9454 Accuracy 0.0907
Epoch 1 Batch 750 Loss 4.8638 Accuracy 0.0958
Epoch 1 Batch 800 Loss 4.7822 Accuracy 0.1005
Epoch 1 Batch 850 Loss 4.7079 Accuracy 0.1054
Epoch 1 Batch 900 Loss 4.6353 Accuracy 0.1101
Epoch 1 Batch 950 Loss 4.5656 Accuracy 0.1146
Epoch 1 Batch 1000 Loss 4.5001 Accuracy 0.1188
Epoch 1 Batch 1050 

# Evaluation

In [70]:
def evaluate(inp_sentence):
    inp_sentence = \
        [VOCAB_SIZE_EN-2] + tokenizer_en.encode(inp_sentence) + [VOCAB_SIZE_EN-1]
    enc_input = tf.expand_dims(inp_sentence, axis=0)
    
    output = tf.expand_dims([VOCAB_SIZE_FR-2], axis=0)
    
    for _ in range(MAX_LENGTH):
        predictions = transformer(enc_input, output, False)
        
        prediction = predictions[:, -1:, :]
        
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)
        
        if predicted_id == VOCAB_SIZE_FR-1:
            return tf.squeeze(output, axis=0)
        
        output = tf.concat([output, predicted_id], axis=-1)
        
    return tf.squeeze(output, axis=0)

# Translation

In [71]:
def translate(sentence):
    output = evaluate(sentence).numpy()
    
    predicted_sentence = tokenizer_fr.decode(
        [i for i in output if i < VOCAB_SIZE_FR-2]
    )
    
    print("Input: {}".format(sentence))
    print("Predicted translation: {}".format(predicted_sentence))

In [72]:
translate("This is a really powerful tool!")

Input: This is a really powerful tool!
Predicted translation: C'est un instrument vraiment fort!


In [78]:
translate("She is very kind human being")

Input: She is very kind human being
Predicted translation: Elle est très importante


In [79]:
translate("My Heart beats for her, why?")

Input: My Heart beats for her, why?
Predicted translation: Ma belle abstraction est-elle la bonne?


In [81]:
translate("I am a man.")

Input: I am a man.
Predicted translation: Je suis un homme.


In [85]:
translate("I am in the space.")

Input: I am in the space.
Predicted translation: Je suis dans l'espace de l'affaire.


I ran this only for 10 Epochs and received accuracy of 50%. As per the accuracy It seems to be working fine. But If we have High end GPU machines. We can train the same model with many number of epochs and get a high accuracy. 


Observations :

1. After evaluating in on different sentences, I found word sequence is maintained but meaning of the words are sometime getting different , Probably it is because of the low accuracy.

2. If remove the full stop from the sentence, It gives different words sometime.
so here We can these symbolls are playing role in terms of balancing the translated sentences

3. If our aim is to input long sentences, We need to make sure vocab size does handles the number of words in the sentence.

# Challenges faced in doing this Project


Challenges :


For the first time, I got good accuracy but transformer's result was not satisfying. Whatever input i used to give to the transformers. It kept giving the sentence with same words.

Later I found out that it is because in Scale.product forgot the minus sign that was not supposed to there hence what is did was 

It multiplied the mask by zero instead of minus infinity so Look ahead mask which i built was not applied,  and Decoder allowed itself to have a look at  the future words instead of guessing them
